# Tokenize pre-mRNA sequences

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, Dataset
import multiprocessing as mp
import pandas as pd
import random
import json
import os

In [ ]:
os.chdir('/work/home/rnasys/zhouhanwen/github/LAMAR_baselines/')

In [ ]:
# tokenizer
tokenizer_path = 'tokenizer/RNA-FM/'
model_max_length = 1026
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path, model_max_length=model_max_length)

In [ ]:
def group_texts(examples):
    tokenized_inputs = tokenizer(examples["seq"], truncation=True, max_length=tokenizer.model_max_length)
    tokenized_inputs["labels"] = examples["label"]
    return tokenized_inputs

In [ ]:
# paths of training set and testing set
train_set_path = 'SpliceSitePred/data/RNA-FM/gencode_train.json'
# path of saving file
data_path = 'SpliceSitePred/data/RNA-FM/ss_single_nucleotide'

In [ ]:
with open(train_set_path) as f:
    train_set = json.load(f)

In [ ]:
# This step takes ~30 mins. Please wait a moment. We will speed up this process.
# Not-known bug will report if use: dataset = load_dataset("json", data_files=train_set_path).
# We directly load the data in memory.
dataset = Dataset.from_list(train_set)
data = dataset.map(group_texts, batched=True, num_proc=mp.cpu_count())
data = data.remove_columns(["seq", "label"])
# data = data["train"].train_test_split(test_size=0.001, shuffle=True, seed=42)
data = data.train_test_split(test_size=0.001, shuffle=True, seed=42)
# Not-known bug will report if set num_proc=mp.cpu_count().
data.save_to_disk(data_path, num_proc=1)

In [ ]:
data